Output URI
s3://groups-bucket-dblab-905418150721/group23/outputs/query3/

Χρησιμοποιώντας ως αναφορά τα δεδομένα της απογραφής 2010 για τον πληθυσμό και εκείνα της απογραφής του 2015 για το εισόδημα ανα νοικοκυριό, να υπολογίσετε για κάθε περιοχή του Los Angeles
τα παρακάτω: Το μέσο ετήσιο εισόδημα ανά άτομο και την αναλογία συνολικού αριθμού εγκλημάτων
ανά άτομο. Τα αποτελέσματα να συγκεντρωθούν σε ένα πίνακα.


## Ανάλυση Δεδομένων ##

Αρχικά τυπώνουμε το Census_Blocks_fields.csv για να δούμε τα πεδία που χρειαζόμαστε

In [1]:
from pyspark.sql import SparkSession
from sedona.spark import *
from pyspark.sql.functions import col

# Create spark Session
spark = SparkSession \
    .builder \
    .appName("Block Label Meaning") \
    .getOrCreate()

labels_path = "s3://initial-notebook-data-bucket-dblab-905418150721/2010_Census_Blocks_fields.csv"
labels = spark.read.format('csv').options(header='true').load(labels_path)
labels.show(30, truncate = False)


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3623,application_1732639283265_3579,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+--------+-------------------------------------------------------------------------+
|field        |type    |meaning                                                                  |
+-------------+--------+-------------------------------------------------------------------------+
|BG10         |string  |7-digit block group number (2010)                                        |
|BG10FIP10    |string  |Combination of BG10 and LA_FIP10 (Los Angeles County FIP code)           |
|BG12         |string  |7-digit block group number (2012)                                        |
|CB10         |string  |4-digit census block number                                              |
|CEN_FIP13    |string  |-                                                                        |
|CITY         |string  |Incorporated city name                                                   |
|CITYCOM      |string  |City/Community name label                                                |
|COMM     

Κάνουμε τις εξής παραδοχές και αποφασίζουμε να χρησιμοποιήσουμε τα εξής πεδία:

COMM: Η περιοχή στην οποία ανήκει το μπλοκ. Εν τέλει θα κάνουμε aggregate όλα τα αποτελέσματα βάσει αυτού

HOUSING10: Πόσα occupied σπίτια υπάρχουν στο block. Το χρειαζόμαστε για να υπολογίσουμε το μέσο εισόδημα κάθε κατοίκου. Στο αντίστοιχο csv για το income μας δίνεται average household      income, όχι ανά κάτοικο, οπότε πρέπει να κάνουμε την υπόθεση ότι το total housing που μας δίνεται είναι πόσα σπίτια είναι occupied στο block. Θα ήταν λάθος να θεωρήσουμε το average income στο άλλο csv να είναι average ανά κάτοικο και να κάνουμε απλώς weighted average στους κατοίκους του community βάσει zip-code.

POP_2010: Μας δίνεται ότι αυτός είναι ο πληθυσμός του block. Θα το χρησιμοποιήσουμε για να κάνουμε aggregate τον συνολικό πληθυσμό του community και να μετρήσουμε τα εγκλήματα ανά άτομο και το income ανά άτομο στο community. Δεδομένου ότι κάθε θα έχουμε περισσότερους κατοίκους από ότι housing, περιμένουμε το income per person να είναι μικρότερο από το income per household

ZCTA10: Το χρησιμοποιούμε για ταχυδρομικό κώδικα. Θα κάνουμε join τα block με το household income βάσει zip code για να υπολογίσουμε τα συνολικά χρήματα του zipcode πριν κάνουμε aggregation βάσει community

geometry: Μας δίνει ένα πολύγωνο, το εσωτερικό του οποίου θεωρείται μέσα στο block. Θα το χρησιμοποιήσουμε για να τοποθετήσουμε τα εγκλήματα μέσα σε block. Αφού μετρήσουμε τα εγκλήματα ανα block μπορούμε να κάνουμε aggregation σε community.

Μετράμε τα blocks, τα zipcodes, και τα communities για να ξέρουμε τις τάξεις μεγέθους (και αν μπορούμε να τυπώσουμε ολόκληρο τον τελικό πίνακα)
Βρίσκουμε επίσης το primary key του block για τα queries μετά

In [2]:
from pyspark.sql import SparkSession
from sedona.spark import *
from pyspark.sql.functions import col
import time

# Create spark Session
spark = SparkSession \
    .builder \
    .appName("communities") \
    .getOrCreate()

# Create sedona context
sedona = SedonaContext.create(spark)


census_path = "s3://initial-notebook-data-bucket-dblab-905418150721/2010_Census_Blocks.geojson"
census = sedona.read.format('geojson').option("multiLine", "true").load(census_path).selectExpr("explode(features) as features").select("features.*")
census = census.select([col(f"properties.{col_name}").alias(col_name) for col_name in census.schema["properties"].dataType.fieldNames()] + ["geometry"]) \
            .drop("properties") \
            .drop("type")

census = census.filter(col("geometry").isNotNull())
census.createOrReplaceTempView("census")

query = """ 
        SELECT *
        FROM census
        """
data = spark.sql(query)
row_count = data.count()
print(f"Number of blocks: {row_count}")

query = """
    SELECT BG10, CB10
    FROM census
    GROUP BY BG10, CB10
    """

data = spark.sql(query)
row_count = data.count()
print(f"Number of BG10, CB10: {row_count}")


query = """ 
        SELECT ZCTA10
        FROM census
        GROUP BY ZCTA10
        """

data = spark.sql(query)
row_count = data.count()
print(f"Number of Zip Codes: {row_count}")


query = """
        SELECT COMM
        FROM census
        GROUP BY COMM        
        """

data = spark.sql(query)
row_count = data.count()
print(f"Number of Communities: {row_count}")



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Number of blocks: 109279
Number of BG10, CB10: 109279
Number of Zip Codes: 301
Number of Communities: 323

Παρατηρούμε ότι έχουμε περισσότερα Communities από ότι zipcodes
Αυτό σημαίνει ότι υπάρχουν εγγραφές με το ίδιο zip code και διαφορετικά communities ή ότι υπάρχουν εγγραφές με NULL communities
Δεν μας πειράζει αυτό ενώ θα βγάζουμε το αποτέλεσμα, αφού το zip code το χρειαζόμαστε μόνο για τα βάρη του income, αλλά θα πρέπει να προσέξουμε να μην κάνουμε group by με το zip code.

Παρατηρούμε επίσης ότι το μέγεθος του τελικού πίνακα θα έχει 323 γραμμές. Το χρησιμοποιούμε ως ένα check για το αν έχουμε κάνει λάθος και δεν τυπώνουμε ολόκληρο τον τελικό πίνακα.

Ελέγχουμε επίσης τον αριθμό των zipcodes στο income table

In [3]:
from pyspark.sql import SparkSession
from sedona.spark import *
from pyspark.sql.functions import col
import time

# Create spark Session
spark = SparkSession \
    .builder \
    .appName("QUERY 3") \
    .getOrCreate()

# Create sedona context
sedona = SedonaContext.create(spark)

income_path = "s3://initial-notebook-data-bucket-dblab-905418150721/LA_income_2015.csv"
census_path = "s3://initial-notebook-data-bucket-dblab-905418150721/2010_Census_Blocks.geojson"

income = spark.read.format('csv').options(header='true').load(income_path)
census = sedona.read.format('geojson').option("multiLine", "true").load(census_path).selectExpr("explode(features) as features").select("features.*")

census = census.select([col(f"properties.{col_name}").alias(col_name) for col_name in census.schema["properties"].dataType.fieldNames()] + ["geometry"]) \
            .drop("properties") \
            .drop("type")

census = census.filter(col("geometry").isNotNull())

income.createOrReplaceTempView("income")
census.createOrReplaceTempView("census")

query  =  """
        SELECT `Zip Code`
        FROM income
        GROUP BY `Zip Code`
        """

data = spark.sql(query)
row_count = data.count()
print(f"Number of Zip Codes In income: {row_count}")

query = """
        SELECT ZCTA10
        FROM census
        JOIN income 
        ON ZCTA10 = `Zip Code`
        GROUP BY ZCTA10
       """

data = spark.sql(query)
row_count = data.count()
print(f"Number of Common Zip Codes In income: {row_count}")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Number of Zip Codes In income: 284
Number of Common Zip Codes In income: 283

Παρατηρούμε ότι από τα 301 zip codes στο census, μπορούμε να πάρουμε income μόνο για τα 283 από αυτά. Αυτό οδηγεί αργότερα σε πρόβλημα, καθώς για 2 communities εξαιτίας αυτού δεν μπορούμε να βγάλουμε census αφού κανένα από τα zcta10 που έχουν μέσα τους δεν δίνει το average income του

# Περιγράφουμε τον τρόπο με τον οποίο υπολογίζουμε το query:
- Αρχικά φορτώνουμε τα δεδομένα από τα csv αρχεία και τα διαμορφώνουμε πριν τα φορτώσουμε ως tables στη βάση
- Δημιουργούμε ST_Points για τα εγκλήματα ώστε να μπορούμε να κάνουμε join με τα blocks και να ξέρουμε μέσα σε ποιο block έγινε κάθε έγκλημα
- Κάνουμε το πρώτο join τα εγκλήματα με τα blocks και κάνουμε groupBy community. 
Πλέον ξέρουμε πόσα εγκλήματα έγιναν σε κάθε community. Αν ένα community δεν έχει εγκλήματα, δυστυχώς θα το χάσουμε. Θεωρητικά θα μπορούσαμε να το λύσουμε αυτό κάνοντας **left join**. Επειδή το left join κολλούσε επ άπειρον το κάναμε χειροκίνητα ως εξής:
- Στον αρχικό πίνακα με τα blocks κάνουμε group by community με στήλη *crime_counter* η οποία έχει το literal 0.
- Κάνουμε union το αποτέλεσμα του join με τον νέο πίνακα.
- Κάνουμε groupBy *community* και κρατάμε το μέγιστο των εγκλημάτων.
Έτσι όποιο community είχε εγκλήματα κρατάει το counter του, ενώ τα υπόλοιπα δεν εξαφανίζονται (αν δεν το κάναμε αυτό θα χάναμε 80 communitites)

Θέλουμε τώρα να υπολογίσουμε τον συνολικό πληθυσμό του κάθε community και το average income
- Αρχίζουμε με τον πίνακα census από τον οποίο κρατάμε τα *ZCTA10*, *COMM*, *POP_2010*, *HOUSING10*.
- Αφού το income βασίζεται στο zip code, αλλά τα zip codes επεκείνονται σε πολλά communities, κάνουμε groupBy τη δυάδα (*ZCTA10,COMM*) κρατώντας το άθροισμα των στοιχείων.
- Καθαρίζουμε τα δεδομένα του table *income* για να μπορούμε να τα χρησιμοποιήσουμε σε αριθμητικές πράξεις.
- Κάνουμε join τα entries του πίνακα των περιοχών με τον πίνακα των εσόδων βάσει του *ZCTA10*.
Σε αυτό το σημείο πάλι χάσαμε κάποιες περιοχές για τις οποίες δεν έχουμε πληροφορίες για το average income. Θα κάνουμε την ίδια τεχνική για **left join** με μια διαφορά. Αν για τον πληθυσμό μιας περιοχής (*ZCTA10,COMM*) δεν έχουμε income data, δεν θέλουμε να συμπεριλάβουμε τον πληθυσμό τα average income data (αφού θα τα μειώσει σαν να είχε average income 0) αλλά θα τον συμπεριλάβουμε στα crime data. Για να το κάνουμε αυτό θα κρατήσουμε δύο πληθυσμούς για κάθε row. Αυτός για το income θα είναι ίσος με τον αρχικό πληθυσμό σε περίπτωση που έχουμε income data για το zip code, διαφορετικά θα είναι 0.
- Συμπληρώνουμε τον πίνακα με τα income όπως περιγράψαμε παραπάνω
- Κάνουμε groupBy στο community βγάζοντας κρατώντας το αρχικό population και το average income με τον εξής τύπο
CAST(SUM(med_inc * houses_per_zip_code_in_community) / SUM(population_for_income) AS INT)
Πλέον έχουμε τα income data και τον πληθυσμό.
- Τέλος κάνουμε join με το *crime_per_community* και υπολογίζουμε τα crimes per person με τον εξής τύπο:
F.col("crime_counter") / F.col("population")

In [4]:
from pyspark.sql import SparkSession, functions as F
from sedona.spark import *
from pyspark.sql.functions import col
import time
from pyspark.sql.functions import monotonically_increasing_id, row_number, expr
from pyspark.sql.window import Window

# Create spark Session
spark = SparkSession \
    .builder \
    .appName("QUERY 3") \
    .getOrCreate()

# Create sedona context
sedona = SedonaContext.create(spark)



crime19path = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv"
crime20path = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv"
income_path = "s3://initial-notebook-data-bucket-dblab-905418150721/LA_income_2015.csv"
census_path = "s3://initial-notebook-data-bucket-dblab-905418150721/2010_Census_Blocks.geojson"

crime19 = spark.read.format('csv').options(header='true').load(crime19path)
crime20 = spark.read.format('csv').options(header='true').load(crime20path)
income = spark.read.format('csv').options(header='true').load(income_path)
census = sedona.read.format('geojson').option("multiLine", "true").load(census_path).selectExpr("explode(features) as features").select("features.*")

census = census.select([col(f"properties.{col_name}").alias(col_name) for col_name in census.schema["properties"].dataType.fieldNames()] + ["geometry"]) \
            .drop("properties") \
            .drop("type")

crime19 = crime19.filter((col("LAT").isNotNull()) & (col("LAT") != 0) & (col("LON").isNotNull()) & (col("LON") != 0))
crime20 = crime20.filter((col("LAT").isNotNull()) & (col("LAT") != 0) & (col("LON").isNotNull()) & (col("LON") != 0))

census = census.filter(col("geometry").isNotNull()).filter((col("CITY") == "Los Angeles"))


crime19.createOrReplaceTempView("crime19")
crime20.createOrReplaceTempView("crime20")
income.createOrReplaceTempView("income")
census.createOrReplaceTempView("census")


join_strategy = "MERGE"

# Union of Crime Tables. No join we don't need to time this
crimes = crime19.union(crime20)
crimes = crimes.withColumn("point", expr("ST_Point(LON, LAT)"))
crime_count = crimes.count()
print(f"[1] Created 'crimes' view with {crime_count} total rows.")

crimes.show(10)

census_counter = census.count()
print(f"[2] Added `id` column to census with {census_counter} rows\n")
census.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[1] Created 'crimes' view with 3109880 total rows.
+---------+--------------------+--------------------+--------+-----+---------+-----------+--------+------+--------------------+--------------+--------+--------+------------+---------+--------------------+--------------+--------------------+------+------------+--------+--------+--------+--------+--------------------+--------------------+-------+---------+--------------------+
|    DR_NO|           Date Rptd|            DATE OCC|TIME OCC|AREA |AREA NAME|Rpt Dist No|Part 1-2|Crm Cd|         Crm Cd Desc|       Mocodes|Vict Age|Vict Sex|Vict Descent|Premis Cd|         Premis Desc|Weapon Used Cd|         Weapon Desc|Status| Status Desc|Crm Cd 1|Crm Cd 2|Crm Cd 3|Crm Cd 4|            LOCATION|        Cross Street|    LAT|      LON|               point|
+---------+--------------------+--------------------+--------+-----+---------+-----------+--------+------+--------------------+--------------+--------+--------+------------+---------+----------

In [5]:
# Suppose you have already defined `census` and `crimes`, 
# and your ST_Within UDF or function is available.

join_strategies = ["BROADCAST", "MERGE", "SHUFFLE_HASH", "SHUFFLE_REPLICATE_NL"]

for strategy in join_strategies:
    print("=" * 60)
    print(f"STEP 3: Spatial Join => blocks_with_crime using {strategy}")
    
    # Clear any cached data to avoid reusing previous results
    # spark.catalog.clearCache()
    
    # Record the start time
    start_time = time.time()
    
    # Build the join pipeline with the given hint
    blocks_with_crime = (
        census
        .hint(strategy)
        .join(
            crimes.hint(strategy),
            # The condition for your spatial join
            ST_Within(F.col("point"), F.col("geometry")),
            "inner"
        )
    )
    
    # Show explain plan so we can see the chosen strategy
    print("\nExplain plan:")
    blocks_with_crime.explain()
    
    # Trigger the actual join by counting rows
    blocks_with_crime_count = blocks_with_crime.count()
    
    elapsed = time.time() - start_time
    print(f"\n[3] Spatial join => {blocks_with_crime_count} rows. "
          f"Elapsed: {elapsed:.2f} s for strategy: {strategy}\n")








# Join census with crimes. We need to time this. NOT equijoin + Big Tables
#print("=== STEP 3: Spatial Join => blocks_with_crime ===")
#start_time = time.time()

#blocks_with_crime = (
#    census
#    .hint(join_strategy)
#    .join(
#        crimes.hint(join_strategy),
#        ST_Within(F.col("point"), F.col("geometry")),
#        "inner"
#    )
#)
# Force evaluation (count triggers the join + aggregation)
#blocks_with_crime_count = blocks_with_crime.count()
#elapsed = time.time() - start_time


#blocks_with_crime_count = blocks_with_crime.count()
#print(f"[3] Spatial join => {blocks_with_crime_count} rows. Elapsed: {elapsed:.2f} s\n")



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

STEP 3: Spatial Join => blocks_with_crime using BROADCAST

Explain plan:
== Physical Plan ==
BroadcastIndexJoin point#882: geometry, LeftSide, RightSide, Inner, WITHIN ST_WITHIN(point#882, geometry#718)
:- SpatialIndex geometry#718: geometry, RTREE, false, false
:  +- *(1) Project [features#715.properties.BG10 AS BG10#724, features#715.properties.BG10FIP10 AS BG10FIP10#725, features#715.properties.BG12 AS BG12#726, features#715.properties.CB10 AS CB10#727, features#715.properties.CEN_FIP13 AS CEN_FIP13#728, features#715.properties.CITY AS CITY#729, features#715.properties.CITYCOM AS CITYCOM#730, features#715.properties.COMM AS COMM#731, features#715.properties.CT10 AS CT10#732, features#715.properties.CT12 AS CT12#733, features#715.properties.CTCB10 AS CTCB10#734, features#715.properties.HD_2012 AS HD_2012#735L, features#715.properties.HD_NAME AS HD_NAME#736, features#715.properties.HOUSING10 AS HOUSING10#737L, features#715.properties.LA_FIP10 AS LA_FIP10#738, features#715.properties.O

In [6]:
# We run it once without any hints just to see what the catalyst optimizer will use on its own
# Clear any cached data to ensure a fresh start
spark.catalog.clearCache()

print("=== STEP 3: Spatial Join => blocks_with_crime (no hint) ===")
start_time = time.time()

# Build the join pipeline without a hint
blocks_with_crime_no_hint = (
    census
    .join(
        crimes,
        ST_Within(F.col("point"), F.col("geometry")),
        "inner"
    )
)

# Show explain plan so we can see what strategy Spark chooses
print("\nExplain plan for no-hint join:")
blocks_with_crime_no_hint.explain()

# Trigger the actual join by counting rows
blocks_with_crime_count_no_hint = blocks_with_crime_no_hint.count()

elapsed = time.time() - start_time
print(f"\nSpatial join => {blocks_with_crime_count_no_hint} rows. "
      f"Elapsed: {elapsed:.2f} s (no hint)\n")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

=== STEP 3: Spatial Join => blocks_with_crime (no hint) ===

Explain plan for no-hint join:
== Physical Plan ==
RangeJoin geometry#718: geometry, point#882: geometry, CONTAINS
:- *(1) Project [features#715.properties.BG10 AS BG10#724, features#715.properties.BG10FIP10 AS BG10FIP10#725, features#715.properties.BG12 AS BG12#726, features#715.properties.CB10 AS CB10#727, features#715.properties.CEN_FIP13 AS CEN_FIP13#728, features#715.properties.CITY AS CITY#729, features#715.properties.CITYCOM AS CITYCOM#730, features#715.properties.COMM AS COMM#731, features#715.properties.CT10 AS CT10#732, features#715.properties.CT12 AS CT12#733, features#715.properties.CTCB10 AS CTCB10#734, features#715.properties.HD_2012 AS HD_2012#735L, features#715.properties.HD_NAME AS HD_NAME#736, features#715.properties.HOUSING10 AS HOUSING10#737L, features#715.properties.LA_FIP10 AS LA_FIP10#738, features#715.properties.OBJECTID AS OBJECTID#739L, features#715.properties.POP_2010 AS POP_2010#740L, features#715.

In [7]:
crime_per_community = (
    blocks_with_crime.groupBy("COMM")
    .agg(F.count("*").alias("crime_counter"))
)

crime_per_community_count = crime_per_community.count()
print(f"[4] Aggregation of all crimes in communities. {crime_per_community_count} Rows \n")
crime_per_community.show(10)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[4] Aggregation of all crimes in communities. 247 Rows 

+--------------------+-------------+
|                COMM|crime_counter|
+--------------------+-------------+
|         Culver City|         1390|
|Rosewood/East Gar...|          175|
|      Toluca Terrace|          300|
|        Elysian Park|         5714|
|            Longwood|         3074|
|         Pico Rivera|            2|
|              Malibu|            1|
|       Green Meadows|        23975|
|    Cadillac-Corning|         4445|
|            Mid-city|        11574|
+--------------------+-------------+
only showing top 10 rows

In [8]:
no_crime_community = (
    census.groupBy("COMM")
)

no_crime_community = census.groupBy("COMM").agg(
    F.lit(0).alias("crime_counter")
)

crime_per_community = crime_per_community.union(no_crime_community)

crime_per_community = crime_per_community.groupBy("COMM").agg(
    F.max("crime_counter").alias("crime_counter")
)

crime_per_community_counter = crime_per_community.count()
print(f"Crime per community rows: {crime_per_community_counter}\n")
crime_per_community.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Crime per community rows: 323

+--------------------+-------------+
|                COMM|crime_counter|
+--------------------+-------------+
|         Culver City|         1390|
|     North Lancaster|            0|
|Rosewood/East Gar...|          175|
|East Rancho Domin...|            0|
|      Toluca Terrace|          300|
|        Elysian Park|         5714|
|            Longwood|         3074|
|         Pico Rivera|            2|
|              Malibu|            1|
|       Green Meadows|        23975|
+--------------------+-------------+
only showing top 10 rows

In [9]:
# Aggregating population and income stats
blocks = (
    census
    .select(
        "ZCTA10",
        "COMM",
        "POP_2010",
        "HOUSING10"
    )
)
# Force evaluation
blocks_count = blocks.count()
print(f"[4] blocks => {blocks_count} rows.\n")

blocks.show(10)

# ------------------------------------------------------------------------------
# STEP 5: Aggregate final_blocks => zip_pop_crime_all
# ------------------------------------------------------------------------------

zip_pop_all = (
    blocks
    .groupBy("ZCTA10", "COMM")
    .agg(
        F.sum("POP_2010").alias("population"),
        F.sum("HOUSING10").alias("houses_per_zip_code_in_community")
    )
)

zip_pop_all_count = zip_pop_all.count()
print(f"[6] zip_pop_all => {zip_pop_all_count} rows.\n")
zip_pop_all.show(10)




# Formatting income, we don't time this
print("=== STEP 7: norm_inc (cleaned income data) ===")
norm_inc = (
    income
    .select(
        F.col("Zip Code").alias("zip_code"),
        F.regexp_replace(
            F.regexp_replace(F.col("Estimated Median Income"), "\\$", ""),
            ",", ""
        ).cast("int").alias("med_inc")
    )
)

norm_inc_count = norm_inc.count()
print(f"[7] norm_inc => {norm_inc_count} rows.\n")
norm_inc.show(10)



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[4] blocks => 109279 rows.

+------+--------------------+--------+---------+
|ZCTA10|                COMM|POP_2010|HOUSING10|
+------+--------------------+--------+---------+
| 93243|West Antelope Valley|       0|        0|
| 93243|West Antelope Valley|       0|        0|
| 93243|West Antelope Valley|       0|        0|
|      |       South Edwards|       0|        0|
|      |       South Edwards|       0|        0|
| 93243|West Antelope Valley|       5|        5|
|      |       South Edwards|       0|        0|
| 90732|           San Pedro|      69|       26|
| 91789|         Diamond Bar|      79|       26|
| 90275| Rancho Palos Verdes|       0|        0|
+------+--------------------+--------+---------+
only showing top 10 rows

[6] zip_pop_all => 1076 rows.

+------+--------------------+----------+--------------------------------+
|ZCTA10|                COMM|population|houses_per_zip_code_in_community|
+------+--------------------+----------+--------------------------------+
| 90211

In [10]:
join_strategies = ["BROADCAST", "MERGE", "SHUFFLE_HASH", "SHUFFLE_REPLICATE_NL"]

for strategy in join_strategies:
    print("============================================================")
    print(f"=== STEP 7: Join zip_pop_all & norm_inc => final_result, Strategy: {strategy} ===")
    
    # Clear any cached data so previous strategies don't affect this one
    spark.catalog.clearCache()
    
    start_time = time.time()
    
    # Apply hints for both DataFrames so Spark respects the join strategy
    joined_data = (
        zip_pop_all
        .hint(strategy)
        .join(
            norm_inc.hint(strategy),
            zip_pop_all.ZCTA10 == norm_inc.zip_code,
            "inner"
        )
    )
    
    # Show the explain plan to confirm which strategy Spark actually uses
    print("\nExplain plan:")
    joined_data.explain()
    
    # Force execution of the join by counting rows
    joined_data_counter = joined_data.count()
    
    join_time = time.time() - start_time
    print(f"\nJoin completed in {join_time:.2f} seconds with {joined_data_counter} rows using {strategy}.\n")
    
# (Optional) Show a few rows
joined_data.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

=== STEP 7: Join zip_pop_all & norm_inc => final_result, Strategy: BROADCAST ===

Explain plan:
== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [ZCTA10#748], [zip_code#2639], Inner, BuildRight, false
   :- HashAggregate(keys=[ZCTA10#748, COMM#731], functions=[sum(POP_2010#740L), sum(HOUSING10#737L)], schema specialized)
   :  +- Exchange hashpartitioning(ZCTA10#748, COMM#731, 1000), ENSURE_REQUIREMENTS, [plan_id=3943]
   :     +- HashAggregate(keys=[ZCTA10#748, COMM#731], functions=[partial_sum(POP_2010#740L), partial_sum(HOUSING10#737L)], schema specialized)
   :        +- Project [features#715.properties.ZCTA10 AS ZCTA10#748, features#715.properties.COMM AS COMM#731, features#715.properties.POP_2010 AS POP_2010#740L, features#715.properties.HOUSING10 AS HOUSING10#737L]
   :           +- Filter (isnotnull(features#715.geometry) AND isnotnull(features#715.properties.ZCTA10))
   :              +- Generate explode(features#707), false, [features#715]
   :    

In [11]:
import time

# Clear any cached data so previous runs don't affect this one
spark.catalog.clearCache()

print("============================================================")
print("=== STEP 7: Join zip_pop_crime_all & norm_inc => final_result, No Hint ===")

start_time = time.time()

# Perform the join without any hint
joined_data_no_hint = (
    zip_pop_all
    .join(
        norm_inc,
        zip_pop_all.ZCTA10 == norm_inc.zip_code,
        "inner"
    )
)

# Show the explain plan to see what strategy Spark chooses automatically
print("\nExplain plan (no hint):")
joined_data_no_hint.explain()

# Force execution of the join by counting rows
joined_data_no_hint_count = joined_data_no_hint.count()

join_time = time.time() - start_time
print(f"\nJoin completed in {join_time:.2f} seconds "
      f"with {joined_data_no_hint_count} rows (no hint).\n")

# (Optional) show a few rows
joined_data_no_hint.show(10)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

=== STEP 7: Join zip_pop_crime_all & norm_inc => final_result, No Hint ===

Explain plan (no hint):
== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [ZCTA10#748], [zip_code#2639], Inner, BuildRight, false
   :- HashAggregate(keys=[ZCTA10#748, COMM#731], functions=[sum(POP_2010#740L), sum(HOUSING10#737L)], schema specialized)
   :  +- Exchange hashpartitioning(ZCTA10#748, COMM#731, 1000), ENSURE_REQUIREMENTS, [plan_id=5698]
   :     +- HashAggregate(keys=[ZCTA10#748, COMM#731], functions=[partial_sum(POP_2010#740L), partial_sum(HOUSING10#737L)], schema specialized)
   :        +- Project [features#715.properties.ZCTA10 AS ZCTA10#748, features#715.properties.COMM AS COMM#731, features#715.properties.POP_2010 AS POP_2010#740L, features#715.properties.HOUSING10 AS HOUSING10#737L]
   :           +- Filter (isnotnull(features#715.geometry) AND isnotnull(features#715.properties.ZCTA10))
   :              +- Generate explode(features#707), false, [features#715]
   :

In [12]:
# copy the population for the income (we will make zero anything without income data)
joined_data = joined_data.withColumn("population_for_income", joined_data.population)
# Drop the zip_code column
joined_data = joined_data.drop("zip_code")


# Add the med_inc and population_for_income to the zip_pop_crime_all to do the left join
zip_pop_all = zip_pop_all.withColumn("med_inc", F.lit(0)) \
                                     .withColumn("population_for_income", F.lit(0))

# Perform the union of the two DataFrames
unioned_data = joined_data.union(zip_pop_all)

# Get the column names to group by (all columns except "population_for_income" and "med_inc")
group_by_columns = ["ZCTA10", "COMM","population","houses_per_zip_code_in_community"]

# Perform the grouping and aggregation
income_data = unioned_data.groupBy(*group_by_columns).agg(
    F.max("population_for_income").alias("population_for_income"),
    F.max("med_inc").alias("med_inc")
)

income_data_counter = income_data.count()
print(f"Unioned data rows: {income_data_counter}\n")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Unioned data rows: 1076

In [13]:
income_result = income_data.groupBy("COMM").agg(
    F.expr("CAST(SUM(med_inc * houses_per_zip_code_in_community) / SUM(population_for_income) AS INT)").alias("average_income"),
    F.expr("SUM(population)").alias("population")
)

income_result_counter = income_result.count()
print(f"Income result table rows: {income_result_counter}\n")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Income result table rows: 323

In [14]:
join_strategies = ["BROADCAST", "MERGE", "SHUFFLE_HASH", "SHUFFLE_REPLICATE_NL"]

for strategy in join_strategies:
    print("====================================================")
    print(f"=== Join income_result & crime_per_community => all_join, Strategy: {strategy} ===")
    
    # Clear any cached data so previous strategies don't affect this run
    spark.catalog.clearCache()
    
    start_time = time.time()
    
    all_join = (
        income_result.hint(strategy)
        .join(
            crime_per_community.hint(strategy),
            on="COMM",
            how="inner"
        )
    )
    
    # Print the physical plan to confirm which strategy Spark uses
    print("\nExplain plan:")
    all_join.explain()
    
    # Trigger the actual join and measure time
    all_join_counter = all_join.count()
    join_time = time.time() - start_time
    
    print(f"\nJoin completed in {join_time:.2f} seconds with {all_join_counter} rows using {strategy}.\n")
    
# (Optional) Show some result rows
all_join.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

=== Join income_result & crime_per_community => all_join, Strategy: BROADCAST ===

Explain plan:
== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [COMM#731, average_income#3046, population#3047L, crime_counter#2482L]
   +- BroadcastHashJoin [COMM#731], [COMM#3087], Inner, BuildLeft, false
      :- BroadcastExchange HashedRelationBroadcastMode(List(input[0, string, true]),false), [plan_id=7430]
      :  +- HashAggregate(keys=[COMM#731], functions=[sum((cast(med_inc#2992 as bigint) * houses_per_zip_code_in_community#2586L)), sum(population_for_income#2990L), sum(population#2584L)], schema specialized)
      :     +- Exchange hashpartitioning(COMM#731, 1000), ENSURE_REQUIREMENTS, [plan_id=7418]
      :        +- HashAggregate(keys=[COMM#731], functions=[partial_sum((cast(med_inc#2992 as bigint) * houses_per_zip_code_in_community#2586L)), partial_sum(population_for_income#2990L), partial_sum(population#2584L)], schema specialized)
      :           +- HashAggregate(keys=[

In [15]:
start_time = time.time()
all_join =  income_result.join(crime_per_community, on="COMM", how="inner")

all_join_counter = all_join.count()
all_join.explain()
# Show the result (optional)
join_time = time.time() - start_time
print(f"Join completed in {join_time:.2f} seconds with {all_join_counter} rows.\n")
all_join.show(10)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [COMM#731, average_income#3046, population#3047L, crime_counter#2482L]
   +- SortMergeJoin [COMM#731], [COMM#3609], Inner
      :- Sort [COMM#731 ASC NULLS FIRST], false, 0
      :  +- HashAggregate(keys=[COMM#731], functions=[sum((cast(med_inc#2992 as bigint) * houses_per_zip_code_in_community#2586L)), sum(population_for_income#2990L), sum(population#2584L)], schema specialized)
      :     +- Exchange hashpartitioning(COMM#731, 1000), ENSURE_REQUIREMENTS, [plan_id=15881]
      :        +- HashAggregate(keys=[COMM#731], functions=[partial_sum((cast(med_inc#2992 as bigint) * houses_per_zip_code_in_community#2586L)), partial_sum(population_for_income#2990L), partial_sum(population#2584L)], schema specialized)
      :           +- HashAggregate(keys=[ZCTA10#748, COMM#731, population#2584L, houses_per_zip_code_in_community#2586L], functions=[max(population_for_income#2938L), max(med_inc#2640)], schema specialized)
      : 

In [16]:
# Create the final table with the desired columns and calculations
final_table = all_join.select(
    "COMM",
    "average_income",
    (F.col("crime_counter") / F.col("population")).alias("crimes_per_person")
)

final_table_counter = final_table.count()
print(f"The final table has {final_table_counter} rows\n")
# Show the result (optional)
final_table.show(30)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

The final table has 323 rows

+--------------------+--------------+--------------------+
|                COMM|average_income|   crimes_per_person|
+--------------------+--------------+--------------------+
|         Culver City|         33650| 0.03574827045238279|
|     North Lancaster|         20102|                 0.0|
|Rosewood/East Gar...|         16211| 0.15034364261168384|
|East Rancho Domin...|          8832|                 0.0|
|      Toluca Terrace|         20167| 0.23059185242121444|
|        Elysian Park|         13871|   1.084868046326182|
|            Longwood|         13420|   0.730166270783848|
|         Pico Rivera|         15157|3.177528518318452E-5|
|              Malibu|         67135|7.908264136022143E-5|
|       Green Meadows|          8027|  1.2095757025377125|
|    Hacienda Heights|         24046|                 0.0|
|    Cadillac-Corning|         19572|  0.6669167291822956|
|  West Puente Valley|         12533|                 0.0|
|          Montebello|    